TODO: LAB NIEUKOŃCZONY
https://www.datacamp.com/tutorial/comprehensive-introduction-graph-neural-networks-gnns-tutorial
https://towardsdatascience.com/graph-convolutional-networks-introduction-to-gnns-24b3f60d6c95

In [140]:
!pip install torch_geometric

In [221]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv, GINConv, GATConv
from sklearn.manifold import TSNE

from torch_geometric.data import DataLoader
import networkx as nx
import random
import matplotlib.pyplot as plt
from torch_geometric.utils.convert import to_networkx


In [188]:
def check_dataset(dataset):
  print(f'Dataset: {dataset}:')
  print('======================')
  print(f'Number of graphs: {len(dataset)}')
  print(f'Number of features: {dataset.num_features}')
  print(f'Number of classes: {dataset.num_classes}')

  data = dataset[0]  # Get the first graph object.
  print(data)


def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

def draw_true_graph_texas(G):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx_nodes(G, pos, node_color=data.y, cmap='Set1', node_size=500)
    nx.draw_networkx_edges(G, pos)
    plt.title("True Graph")
    plt.show()

def draw_graph_predict_texas(G, pred):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)
    nx.draw_networkx_nodes(G, pos, node_color=pred, cmap='Set1', node_size=500)
    nx.draw_networkx_edges(G, pos)
    plt.title("Predicted Graph")
    plt.show()

In [189]:
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import KarateClub
from torch_geometric.datasets import Planetoid

In [190]:
datasetTU = TUDataset(root='data/TUDataset', name='MUTAG')
check_dataset(datasetTU)

Dataset: MUTAG(188):
Number of graphs: 188
Number of features: 7
Number of classes: 2
Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])


In [191]:
datasetPlanetoid = Planetoid(root='/tmp/Cora', name='Cora')
check_dataset(datasetPlanetoid)

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [192]:
datasetKB = WebKB(root='.', name='texas')
check_dataset(datasetKB)

Dataset: texas():
Number of graphs: 1
Number of features: 1703
Number of classes: 5
Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])


In [223]:
class GCN(torch.nn.Module):
    def __init__(self, dataset):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GIN(torch.nn.Module):
    def __init__(self, dataset):
        super(GIN, self).__init__()
        self.conv1 = GINConv(dataset.num_node_features, 16)
        self.conv2 = GINConv(16, dataset.num_classes)
        self.layers = nn.ModuleList([self.conv1, self.conv2])

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class GAT(torch.nn.Module):
    def __init__(self, dataset):
        super(GAT, self).__init__()
        self.conv1 = GATConv(dataset.num_node_features, 16, heads=8)
        self.conv2 = GATConv(16 * 8, dataset.num_classes, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [218]:
def training_loop(dataset, model, criterion, optimizer, skip_visualisation):
    # Load the first graph from the dataset
    data = dataset[0]

    # Randomly select 10 nodes as training nodes
    data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    data.train_mask[random.sample(range(data.num_nodes), 10)] = 1

    # Convert the data to a networkx graph
    G = to_networkx(data)

    # Training loop with classification output for each epoch
    for epoch in range(20):
        print(f"---------------------------- EPOCH: {epoch}---------------------------------------")
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        model.eval()
        pred = model(data.x, data.edge_index).argmax(dim=1)
        acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean()
        print(f'Epoch {epoch}: Train Accuracy: {acc.item() * 100:.4f}%')

        # Draw the true graph and the predicted graph
        if not skip_visualisation:
          draw_true_graph_texas(to_networkx(data))
          draw_graph_predict_texas(to_networkx(data), pred)

        print("-------------------------------------------------------------------")


    # Set the model to evaluation mode and display the results
    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=1)
    acc = (pred[data.train_mask] == data.y[data.train_mask]).float().mean()
    print(f'Final Training Accuracy: {acc.item() * 100:.4f}')

    # Display the classification results for all nodes
    pred = model(data.x, data.edge_index).argmax(dim=1)
    print(f'Predicted Class Labels: {pred}')

In [219]:
skip_visualisation = True

In [215]:
model_GCN = GCN(datasetKB)
criterion_GCN = torch.nn.NLLLoss()
optimizer_GCN = torch.optim.Adam(model_GCN.parameters(), lr=0.001)

training_loop(datasetKB, model_GCN, criterion_GCN, optimizer_GCN, skip_visualisation)

---------------------------- EPOCH: 0---------------------------------------
Epoch 0: Train Accuracy: 20.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 1---------------------------------------
Epoch 1: Train Accuracy: 40.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 2---------------------------------------
Epoch 2: Train Accuracy: 60.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 3---------------------------------------
Epoch 3: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 4---------------------------------------
Epoch 4: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 5---------------------------------------
Epoch 5: Train Accuracy: 70.

In [225]:
model_GIN = GIN(datasetKB)
print(list(model_GIN.parameters()))
criterion_GIN = torch.nn.NLLLoss()
optimizer_GIN = torch.optim.Adam(model_GIN.parameters(), lr=0.001)

training_loop(datasetKB, model_GIN, criterion_GIN, optimizer_GIN, skip_visualisation)

[]


ValueError: optimizer got an empty parameter list

In [216]:
model = GAT(datasetKB)
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

training_loop(datasetKB, model, criterion, optimizer, skip_visualisation)

---------------------------- EPOCH: 0---------------------------------------
Epoch 0: Train Accuracy: 60.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 1---------------------------------------
Epoch 1: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 2---------------------------------------
Epoch 2: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 3---------------------------------------
Epoch 3: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 4---------------------------------------
Epoch 4: Train Accuracy: 70.0000%
-------------------------------------------------------------------
---------------------------- EPOCH: 5---------------------------------------
Epoch 5: Train Accuracy: 70.

# GIN vs GAT vs GCN